# Book Recommendation System

## Import all Packages 

In [1]:
import pandas as pd
import numpy as np
import pandas as pd
import nltk
import json
import re
import csv
from tqdm import tqdm
from textblob import TextBlob

## Open and try to read all contents

In [2]:
pd.set_option('display.max_colwidth', 300)
data = []
with open("../BOOK_DATASET/booksummaries.txt", 'r', encoding='utf-8') as f:
    reader = csv.reader(f, dialect='excel-tab')
    for row in tqdm(reader):
        data.append(row)


16559it [00:01, 16199.64it/s]


## We append a all attributes in csv

In [3]:
book_index = []
book_id = []
book_author = []
book_name = []
summary = []
genre = []
a = 1
for i in tqdm(data):
    book_index.append(a)
    a = a+1
    book_id.append(i[0])
    book_name.append(i[2])
    book_author.append(i[3])
    genre.append(i[5])
    summary.append(i[6])

df = pd.DataFrame({'Index': book_index, 'ID': book_id, 'BookTitle': book_name, 'Author': book_author,
                       'Genre': genre, 'Summary': summary})
df.head()

100%|████████████████████████████████████████████████████████████████████████| 16559/16559 [00:00<00:00, 888345.04it/s]


,Index,ID,BookTitle,Author,Genre,Summary
0,1,620,Animal Farm,George Orwell,"{""/m/016lj8"": ""Roman \u00e0 clef"", ""/m/06nbt"": ""Satire"", ""/m/0dwly"": ""Children's literature"", ""/m/014dfn"": ""Speculative fiction"", ""/m/02xlf"": ""Fiction""}","Old Major, the old boar on the Manor Farm, calls the animals on the farm for a meeting, where he compares the humans to parasites and teaches the animals a revolutionary song, 'Beasts of England'. When Major dies, two young pigs, Snowball and Napoleon, assume command and turn his dream into a p..."
1,2,843,A Clockwork Orange,Anthony Burgess,"{""/m/06n90"": ""Science Fiction"", ""/m/0l67h"": ""Novella"", ""/m/014dfn"": ""Speculative fiction"", ""/m/0c082"": ""Utopian and dystopian fiction"", ""/m/06nbt"": ""Satire"", ""/m/02xlf"": ""Fiction""}","Alex, a teenager living in near-future England, leads his gang on nightly orgies of opportunistic, random ""ultra-violence."" Alex's friends (""droogs"" in the novel's Anglo-Russian slang, Nadsat) are: Dim, a slow-witted bruiser who is the gang's muscle; Georgie, an ambitious second-in-command; and..."
2,3,986,The Plague,Albert Camus,"{""/m/02m4t"": ""Existentialism"", ""/m/02xlf"": ""Fiction"", ""/m/0pym5"": ""Absurdist fiction"", ""/m/05hgj"": ""Novel""}","The text of The Plague is divided into five parts. In the town of Oran, thousands of rats, initially unnoticed by the populace, begin to die in the streets. A hysteria develops soon afterward, causing the local newspapers to report the incident. Authorities responding to public pressure order t..."
3,4,1756,An Enquiry Concerning Human Understanding,David Hume,,"The argument of the Enquiry proceeds by a series of incremental steps, separated into chapters which logically succeed one another. After expounding his epistemology, Hume explains how to apply his principles to specific topics. In the first section of the Enquiry, Hume provides a rough introdu..."
4,5,2080,A Fire Upon the Deep,Vernor Vinge,"{""/m/03lrw"": ""Hard science fiction"", ""/m/06n90"": ""Science Fiction"", ""/m/014dfn"": ""Speculative fiction"", ""/m/01hmnh"": ""Fantasy"", ""/m/02xlf"": ""Fiction""}","The novel posits that space around the Milky Way is divided into concentric layers called Zones, each being constrained by different laws of physics and each allowing for different degrees of biological and technological advancement. The innermost, the ""Unthinking Depths"", surrounds the galacti..."


In [4]:
def clean_summary(text):
    text = re.sub("\'", "", text)
    text = re.sub("[^a-zA-Z]"," ",text)
    text = ' '.join(text.split())
    text = text.lower()
    return text

df['clean_summary'] = df['Summary'].apply(lambda x: clean_summary(x))
df.head(2)

,Index,ID,BookTitle,Author,Genre,Summary,clean_summary
0,1,620,Animal Farm,George Orwell,"{""/m/016lj8"": ""Roman \u00e0 clef"", ""/m/06nbt"": ""Satire"", ""/m/0dwly"": ""Children's literature"", ""/m/014dfn"": ""Speculative fiction"", ""/m/02xlf"": ""Fiction""}","Old Major, the old boar on the Manor Farm, calls the animals on the farm for a meeting, where he compares the humans to parasites and teaches the animals a revolutionary song, 'Beasts of England'. When Major dies, two young pigs, Snowball and Napoleon, assume command and turn his dream into a p...",old major the old boar on the manor farm calls the animals on the farm for a meeting where he compares the humans to parasites and teaches the animals a revolutionary song beasts of england when major dies two young pigs snowball and napoleon assume command and turn his dream into a philosophy t...
1,2,843,A Clockwork Orange,Anthony Burgess,"{""/m/06n90"": ""Science Fiction"", ""/m/0l67h"": ""Novella"", ""/m/014dfn"": ""Speculative fiction"", ""/m/0c082"": ""Utopian and dystopian fiction"", ""/m/06nbt"": ""Satire"", ""/m/02xlf"": ""Fiction""}","Alex, a teenager living in near-future England, leads his gang on nightly orgies of opportunistic, random ""ultra-violence."" Alex's friends (""droogs"" in the novel's Anglo-Russian slang, Nadsat) are: Dim, a slow-witted bruiser who is the gang's muscle; Georgie, an ambitious second-in-command; and...",alex a teenager living in near future england leads his gang on nightly orgies of opportunistic random ultra violence alexs friends droogs in the novels anglo russian slang nadsat are dim a slow witted bruiser who is the gangs muscle georgie an ambitious second in command and pete who mostly pla...


In [5]:
book = input("Please enter a book title: ")
clean_book = book.strip().capitalize()

clean_book_titles = [title for title in df['BookTitle'].unique()]

similar_books = []
for title in clean_book_titles:
    if clean_book in title:
        similar_books.append(title)

if len(similar_books) > 0:
    print("Do you mean these books:")
    for i, book in enumerate(similar_books):
        print(f"{i+1}. {book}")
    option = int(input("Please enter the corresponding option number: "))
    book_title = similar_books[option-1]
    
    # Get the summary of the selected book
    book_summary = df[df['BookTitle'] == book_title]['Summary'].iloc[0]
    
else:
    print("No similar books found.")
    book_title = None
    book_summary = None




Please enter a book title: die
Do you mean these books:
1. Lord Edgware Dies
2. He Died With A Felafel In His Hand
3. Win, Lose or Die
4. Veronika Decides to Die
5. The King Must Die
6. The Way Some People Die
7. Malone Dies
8. Generals Die in Bed
9. Spock Must Die!
10. Heroes Die
11. When Jonathan Died
12. Die Vecna Die!
13. More Die of Heartbreak
14. The Last Camel Died at Noon
15. Some Lie And Some Die
16. The Best Man to Die
17. Vanity Dies Hard
18. Or Die Trying
19. Die Dame im Chinchilla
20. If I Die in a Combat Zone: Box Me Up and Ship Me Home
21. Live and Let Die
22. See Delphi and Die
23. And Chaos Died
24. Die Feuerzangenbowle
25. And Then You Die
26. Die Trying
27. Dies the Fire
28. Die neuen Leiden des jungen W.
29. Now I Can Die in Peace
30. A Right to Die
31. Die Jungfrau von Orleans
32. The Sweet Dove Died
33. She Died a Lady
34. 1st to Die
35. A Summer to Die
36. John Dies at the End
37. Every Man Dies Alone
38. Before I Die
39. Tamburlaine Must Die
40. Time Travelers N

In [6]:
import spacy

# Load the pre-trained model
nlp = spacy.load("en_core_web_lg")


# Process the text with spaCy
doc = nlp(book_summary)

# Extract person entities
people = [ent.text for ent in doc.ents if ent.label_ == "PERSON"]

# Select the most frequently occurring person as the main character
if people:
    main_character = max(set(people), key=people.count)
    print("The main character is:", main_character)
else:
    print("No person entities were found in the text.")


The main character is: Lindsay


In [ ]:
import spacy
import random

score = 0
nlp = spacy.load("en_core_web_sm")

doc = nlp(book_summary)

nouns = [token.text for token in doc if token.pos_ == "NOUN"]
verbs = [token.text for token in doc if token.pos_ == "VERB"]

def generate_mcq():
    noun = random.choice(nouns)
    verb = random.choice(verbs)
    question = f"What {noun} did {verb} in {book_title}?"
    options = random.sample(nouns + verbs, 4)
    options.append(noun)
    random.shuffle(options)
    return (question, options, noun)

# Generate 5 MCQs for the user to answer
    
for i in range(5):
    # Generate a MCQ
    question, options, noun = generate_mcq()
    print(f"{i+1}. {question}")
    for j, option in enumerate(options):
        print(f"   {j+1}. {option}")
    answer = input("Your answer (Enter the option number): ")
    if int(answer) == options.index(noun) + 1:
        print("Correct!")
        score+=1
    else:
        print(f"Incorrect. The correct answer is {noun}.")
        
if main_character:
    main_character_answer = input(f"Who is the main character of {book_title}? ")
    if main_character_answer.strip().lower() == main_character.lower():
        print("Correct!")
        score+=1
    else:
        print(f"Incorrect. The correct answer is {main_character}.")

print (f"Your final score is {score} ")

C:\Users\BALAJI K\anaconda3\lib\site-packages\spacy\util.py:877: UserWarning: [W095] Model 'en_core_web_sm' (3.0.0) was trained with spaCy v3.0 and may not be 100% compatible with the current version (3.4.2). If you see errors or degraded performance, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)


1. What room did opens in 1st to Die?
   1. case
   2. has
   3. calls
   4. room
   5. struggles


In [ ]:
if score>=4:
    df = pd.read_csv("../BOOK_DATASET/Books.csv")
    isbn = df.loc[df["Book-Title"] == book_title, "ISBN"].values[0]
    print(f"The ISBN ID of {book_title} is {isbn}.")
    user_id = input("Enter your ID: ")
    rating = input("Enter your rating (out of 5): ")
    ratings_df = pd.read_csv("../BOOK_DATASET/Rate.csv")
    new_rating = {"USER_ID": user_id, "ISBN": isbn, "Book_Rating": rating}
    ratings_df = ratings_df.append(new_rating, ignore_index=True)
    ratings_df.to_csv("Ratee.csv", index=False)
    print("Your rating has been added")